In [ ]:
!pip install geopandas
!pip install pyshp
!pip install shapely
!pip install geoplot
!pip install altair
!pip install vega
!pip install gpdvega

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.cloud import bigquery
from google.colab import auth

import pandas as pd
import json
import geopandas as gpd
from shapely import wkt
from shapely import geometry
import matplotlib.pyplot as plt
import altair as alt
import gpdvega
import vega
import time

In [ ]:
auth.authenticate_user()
print('Authenticated')
PROJECT = '<YOUR PROJECT>' 
PROJECT = 'fiery-pipe-353815'

Authenticated


In [ ]:
%%bigquery storm_data_Jan_2019 --project $PROJECT

SELECT  *
FROM `bigquery-public-data.noaa_historic_severe_storms.storms_2019` 
WHERE 
EXTRACT(MONTH FROM event_begin_time)=1

In [ ]:
# # from google.cloud import bigquery
# # client = bigquery.Client()
# # bucket_name = 'my-bucket'
# project = "fiery-pipe-353815"
# dataset_id = "bigquery-public-data.noaa_historic_severe_storms"

# destination_uri = "gs://{}/{}".format(bucket_name, "storm.csv")

# extract_job = client.extract_table(

#     destination_uri,
#     # Location must match that of the source table.

# )  # API request
# extract_job.result()  # Waits for job to complete.

# print(
#     "Exported {}:{}.{} to {}".format(project, dataset_id, destination_uri)
# )

NameError: ignored

In [ ]:
# import mysql.connector
# import pandas as pd 
# my_connect = mysql.connector.connect(
#       host="localhost",
#       user="root",
#       passwd="test",
#       database="my_tutorial"
#     )
# ####### end of connection ####
# sql="SELECT * FROM student "
# my_data = pd.read_sql(sql,my_connect )
# my_data.to_csv('D:\my_file.csv')

In [ ]:
# Create dataset
client = bigquery.Client(project = PROJECT)
dataset_name = "storm"
dataset_id = "{}.{}".format(client.project, dataset_name)

# Construct a full Dataset object to send to the API.
dataset = bigquery.Dataset(dataset_id)

# Send the dataset to the API for creation.
# Raises google.api_core.exceptions.Conflict if the Dataset already
# exists within the project.
dataset = client.create_dataset(dataset)  # API request
print("Created dataset {}.{}".format(client.project, dataset.dataset_id))

Created dataset fiery-pipe-353815.storm


In [ ]:
# client.delete_dataset(
#     'storm', delete_contents=True, not_found_ok=True
# ) 
# print("Deleted dataset storm")

Deleted dataset insurance_demo


In [ ]:
%%bigquery --project $PROJECT
SELECT * INTO storm FROM bigquery-public-data.noaa_historic_severe_storms.storms_2019

In [ ]:
%%bigquery --project $PROJECT
SELECT * 
FROM
    `bigquery-public-data.noaa_historic_severe_storms.storms_*`,
    `bigquery-public-data.geo_us_boundaries.zip_codes` AS zipcodes


Executing query with job ID: 8cf10be9-2821-409c-bfd2-9fd1917caf22
Query executing: 1088.67s

In [ ]:
storm.head(10)

NameError: ignored

In [ ]:
storm_data_Jan_2019.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5216 entries, 0 to 5215
Data columns (total 33 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   episode_id          5216 non-null   object        
 1   event_id            5216 non-null   object        
 2   state               5216 non-null   object        
 3   state_fips_code     5216 non-null   object        
 4   event_type          5216 non-null   object        
 5   cz_type             5216 non-null   object        
 6   cz_fips_code        5216 non-null   object        
 7   cz_name             5216 non-null   object        
 8   wfo                 5216 non-null   object        
 9   event_begin_time    5216 non-null   datetime64[ns]
 10  event_timezone      5216 non-null   object        
 11  event_end_time      5216 non-null   datetime64[ns]
 12  injuries_direct     5216 non-null   int64         
 13  injuries_indirect   5216 non-null   int64       

In [ ]:
storm_data_Jan_2019.head(10)

,episode_id,event_id,state,state_fips_code,event_type,cz_type,cz_fips_code,cz_name,wfo,event_begin_time,...,tor_length,tor_width,tor_other_wfo,location_index,event_range,event_azimuth,reference_location,event_latitude,event_longitude,event_point
0,132374,791996,Kentucky,21,hail,C,203,ROCKCASTLE,JKL,2019-01-04 17:29:00,...,None,0,None,1,0.00,N,MT VERNON,37.3500,-84.3500,POINT(-84.35 37.35)
1,132592,793433,Louisiana,22,hail,C,63,LIVINGSTON,LIX,2019-01-19 08:40:00,...,None,0,None,1,0.00,N,WALKER,30.4800,-90.8700,POINT(-90.87 30.48)
2,132609,793476,Texas,48,hail,C,387,RED RIVER,SHV,2019-01-19 00:10:00,...,None,0,None,1,0.15,NNE,AVERY,33.5520,-94.7791,POINT(-94.7791 33.552)
3,132610,793477,Arkansas,5,hail,C,81,LITTLE RIVER,SHV,2019-01-19 00:38:00,...,None,0,None,1,0.45,ENE,FOREMAN,33.7214,-94.3924,POINT(-94.3924 33.7214)
4,132910,795207,Ohio,39,hail,C,93,LORAIN,CLE,2019-01-08 15:20:00,...,None,0,None,1,1.04,W,ELYRIA,41.3700,-82.1200,POINT(-82.12 41.37)
5,132910,795206,Ohio,39,hail,C,35,CUYAHOGA,CLE,2019-01-08 15:26:00,...,None,0,None,1,0.86,NW,(CGF)CUYAHOGA CO ARP,41.5800,-81.4900,POINT(-81.49 41.58)
6,132910,795208,Ohio,39,hail,C,93,LORAIN,CLE,2019-01-08 15:30:00,...,None,0,None,1,0.69,N,LA GRANGE,41.2400,-82.1200,POINT(-82.12 41.24)
7,132910,795211,Ohio,39,hail,C,35,CUYAHOGA,CLE,2019-01-08 15:51:00,...,None,0,None,1,1.48,SSE,STRONGSVILLE ARPT,41.3000,-81.8600,POINT(-81.86 41.3)
8,132910,807693,Ohio,39,hail,C,143,SANDUSKY,CLE,2019-01-08 14:30:00,...,None,0,None,1,1.09,NE,LINDSEY,41.4309,-83.2048,POINT(-83.2048 41.4309)
9,132910,807694,Ohio,39,hail,C,153,SUMMIT,CLE,2019-01-08 15:24:00,...,None,0,None,1,1.12,W,AKRON MAYFIELD ARPT,40.9775,-81.4511,POINT(-81.4511 40.9775)


In [ ]:
%%bigquery storm_data_state --project $PROJECT
# Get Storm data by State for the past 5 years from BigQuery Pulic Dataset
WITH
  storm AS (
  SELECT
    state,
    event_type,
    COUNT(event_id) no_of_storms,
    SUM(damage_property) AS damage_property,
    SUM(damage_crops) AS damage_crops,
    SUM(damage_property + damage_crops) AS damage_cost
  FROM
    `bigquery-public-data.noaa_historic_severe_storms.storms_*`
  WHERE
    EXTRACT(YEAR
    FROM
      event_begin_time)>= EXTRACT(YEAR
    FROM
      CURRENT_DATE())-5
  GROUP BY
    state,
    event_type  )
SELECT
  storm.*,  
  state_fips_code,
  int_point_lat AS state_latitude,
  int_point_lon AS state_longitude
FROM
  storm,
  `bigquery-public-data.geo_us_boundaries.states`
WHERE
  storm.state=state_name


In [ ]:
# Plot no. of storms and cost of damange on US map 

# Get States data from vega-datasets for mapping
from vega_datasets import data
states = alt.topo_feature(data.us_10m.url, feature='states')

# Encode basemap to depict all states
basemap= alt.Chart(states).mark_geoshape(
        stroke='white',
        strokeWidth=1,
        fill='lightgray'
    ).properties(
        width=500,
        height=400
    ).project('albersUsa')

# Encode filled map to depict damage cost by states
storm_data_by_state = storm_data_state.groupby(['state','state_fips_code'])['damage_cost'].sum().reset_index(name='damage_cost')
storm_data_by_state['state_fips_code']=storm_data_by_state['state_fips_code'].str.lstrip("0")
statemap = alt.Chart(states).mark_geoshape(  

).encode(color=alt.Color('damage_cost:Q', scale=alt.Scale(scheme='greenblue'),
                         legend=alt.Legend(title = "Total damage Cost")),
         tooltip=['state:N','damage_cost:Q']         
).transform_lookup(
    lookup='id',
    from_=alt.LookupData(storm_data_by_state, 'state_fips_code', ['damage_cost','state']) 

)

# Encode point map to depict no. of storms by state
pointmap = alt.Chart(storm_data_state).mark_circle().encode(
    longitude='state_longitude:Q',
    latitude='state_latitude:Q',
    size=alt.Size('sum(no_of_storms):Q', title='Number of Storms'),
    color=alt.Color('sum(no_of_storms):Q', scale=alt.Scale(scheme='blueorange'),
                legend=alt.Legend(orient='right')),
    tooltip=['state:N','sum(no_of_storms):Q', 'sum(damage_cost):Q']
)

costmap = basemap + statemap


/usr/local/lib/python3.7/dist-packages/altair/utils/data.py:229: AltairDeprecationWarning: alt.pipe() is deprecated, and will be removed in a future release. Use toolz.curried.pipe() instead.
  AltairDeprecationWarning,


In [ ]:
# Layer the filled and point maps
alt.layer(costmap, pointmap).resolve_legend(
    color="independent",
    size="independent"
).resolve_scale(color="independent")

/usr/local/lib/python3.7/dist-packages/altair/utils/data.py:229: AltairDeprecationWarning: alt.pipe() is deprecated, and will be removed in a future release. Use toolz.curried.pipe() instead.
  AltairDeprecationWarning,


alt.LayerChart(...)

In [ ]:
alt.Chart(storm_data_state).transform_aggregate(
    damage = 'sum(damage_cost)',
    groupby = ['event_type']
).transform_window(
    rank='rank(damage)',
    sort=[alt.SortField('damage', order='descending')]
).transform_filter(
    alt.datum.rank <=10
).mark_bar().encode(
    alt.X('damage:Q', title='Total Damage Cost'), 
    alt.Y('event_type:N', sort='-x'),       
    color=alt.value('green'),    
    tooltip = ['event_type:N','sum(no_of_storms):Q','sum(damage_cost):Q']
)

/usr/local/lib/python3.7/dist-packages/altair/utils/data.py:229: AltairDeprecationWarning: alt.pipe() is deprecated, and will be removed in a future release. Use toolz.curried.pipe() instead.
  AltairDeprecationWarning,


alt.Chart(...)

In [ ]:
alt.Chart(storm_data_state).mark_point().encode(
    x='no_of_storms:Q',
    y='damage_cost:Q',
    color='event_type:N',    
    tooltip = ['state:N','event_type:N','no_of_storms:Q','damage_cost:Q']
).configure_point(
    size=100
)

/usr/local/lib/python3.7/dist-packages/altair/utils/data.py:229: AltairDeprecationWarning: alt.pipe() is deprecated, and will be removed in a future release. Use toolz.curried.pipe() instead.
  AltairDeprecationWarning,


alt.Chart(...)

In [ ]:
storm_data_state_subset = storm_data_state[storm_data_state['state'].isin(['California','Texas','Lousiana','Florida','Georgia'])]
storm_data_state_subset = storm_data_state_subset[storm_data_state_subset['event_type'].isin(['flash flood','wildfire','hurricane','flood','hail','tornado','tropical storm','storm surge/tide','high wind','thunderstorm wind'])]


In [ ]:
alt.Chart(storm_data_state_subset).mark_point().encode(
    x='no_of_storms:Q',
    y='damage_cost:Q',
    color='event_type:N',    
    tooltip = ['state:N','event_type:N','no_of_storms:Q','damage_cost:Q']
).configure_point(
    size=100
)

/usr/local/lib/python3.7/dist-packages/altair/utils/data.py:229: AltairDeprecationWarning: alt.pipe() is deprecated, and will be removed in a future release. Use toolz.curried.pipe() instead.
  AltairDeprecationWarning,


alt.Chart(...)

In [ ]:
%%bigquery storm_data_zipcodes --project $PROJECT
# Storm data by State for the past 5 years

  SELECT
    state,
    zip_code,
    internal_point_lat AS zip_code_latitude,
    internal_point_lon AS zip_code_longitude,
    COUNT(event_id) no_of_storms,
    SUM(damage_property) AS damage_property,
    SUM(damage_crops) AS damage_crops,
    SUM(damage_property + damage_crops) AS damage_cost
  FROM
    `bigquery-public-data.noaa_historic_severe_storms.storms_*`,
    `bigquery-public-data.geo_us_boundaries.zip_codes` AS zipcodes
  WHERE
    EXTRACT(YEAR
    FROM
      event_begin_time)>= EXTRACT(YEAR
    FROM
      CURRENT_DATE())-5
    AND ST_WITHIN(event_point,zip_code_geom)
    AND state IN ('California','Texas','Louisiana','Georgia','Florida')  
    AND event_type IN ('flash flood','wildfire','hurricane','flood','hail','tornado','tropical storm','storm surge/tide','high wind','thunderstorm wind')
  GROUP BY
    state,
    zip_code,
    internal_point_lat,
    internal_point_lon  
  

 


In [ ]:

alt.Chart(storm_data_zipcodes).mark_point().encode(
    x='no_of_storms:Q',
    y='damage_cost:Q',
    color=alt.Color('state:N',scale=alt.Scale(scheme='category10')),
    #size=alt.Size('Hours per Month to Afford a Home:Q'),
    tooltip = ['zip_code:N', 'state:N','no_of_storms:Q','damage_cost:Q']
).configure_point(
    size=100
)

/usr/local/lib/python3.7/dist-packages/altair/utils/data.py:229: AltairDeprecationWarning: alt.pipe() is deprecated, and will be removed in a future release. Use toolz.curried.pipe() instead.
  AltairDeprecationWarning,


alt.Chart(...)

In [ ]:
zmap = alt.Chart(storm_data_zipcodes).mark_circle().encode(
    longitude='zip_code_longitude:Q',
    latitude='zip_code_latitude:Q',
    size=alt.Size('sum(damage_cost):Q', title=''),
    color=alt.Color('sum(damage_cost):Q', scale=alt.Scale(scheme='oranges'),
                legend=alt.Legend(orient='right', title='Damage Cost by Zip Code')),
    tooltip=['state:N','zip_code:N','sum(no_of_storms):Q', 'sum(damage_cost):Q']
)

alt.layer(basemap, zmap).resolve_legend(
    color="independent",
    size="independent"
).resolve_scale(color="independent")

/usr/local/lib/python3.7/dist-packages/altair/utils/data.py:229: AltairDeprecationWarning: alt.pipe() is deprecated, and will be removed in a future release. Use toolz.curried.pipe() instead.
  AltairDeprecationWarning,


alt.LayerChart(...)

In [ ]:
%%bigquery storm_data_zipcodes_20yearsago --project $PROJECT
# Get Storm data by Zip Code for the past 20 years

  SELECT
    state,
    zip_code,
    internal_point_lat AS zip_code_latitude,
    internal_point_lon AS zip_code_longitude,
    COUNT(event_id) no_of_storms,
    SUM(damage_property) AS damage_property,
    SUM(damage_crops) AS damage_crops,
    SUM(damage_property + damage_crops) AS damage_cost
  FROM
    `bigquery-public-data.noaa_historic_severe_storms.storms_*`,
    `bigquery-public-data.geo_us_boundaries.zip_codes` AS zipcodes
  WHERE
    EXTRACT(YEAR
    FROM
      event_begin_time)>= EXTRACT(YEAR
    FROM
      CURRENT_DATE())-25
    AND EXTRACT(YEAR
    FROM
      event_begin_time)<= EXTRACT(YEAR
    FROM
      CURRENT_DATE())-20
    AND ST_WITHIN(event_point,zip_code_geom)
    AND state IN ('California','Texas','Louisiana','Georgia','Florida')  
    AND event_type IN ('flash flood','wildfire','hurricane','flood','hail','tornado','tropical storm','storm surge/tide','high wind','thunderstorm wind')
  GROUP BY
    state,
    zip_code,
    internal_point_lat,
    internal_point_lon  

In [ ]:
zmap_20 = alt.Chart(storm_data_zipcodes_20yearsago).mark_circle().encode(
    longitude='zip_code_longitude:Q',
    latitude='zip_code_latitude:Q',
    size=alt.Size('sum(damage_cost):Q', title=''),
    color=alt.Color('sum(damage_cost):Q', scale=alt.Scale(scheme='teals'),
                legend=alt.Legend(orient='right', title='Damage Cost by Zip Code')),
    tooltip=['state:N','zip_code:N','sum(no_of_storms):Q', 'sum(damage_cost):Q']
)

alt.layer(basemap, zmap_20).resolve_legend(
    color="independent",
    size="independent"
).resolve_scale(color="independent")

/usr/local/lib/python3.7/dist-packages/altair/utils/data.py:229: AltairDeprecationWarning: alt.pipe() is deprecated, and will be removed in a future release. Use toolz.curried.pipe() instead.
  AltairDeprecationWarning,


alt.LayerChart(...)

In [ ]:
# Create dataset
client = bigquery.Client(project = PROJECT)
dataset_name = "insurance_demo"
dataset_id = "{}.{}".format(client.project, dataset_name)

# Construct a full Dataset object to send to the API.
dataset = bigquery.Dataset(dataset_id)

# Send the dataset to the API for creation.
# Raises google.api_core.exceptions.Conflict if the Dataset already
# exists within the project.
dataset = client.create_dataset(dataset)  # API request
print("Created dataset {}.{}".format(client.project, dataset.dataset_id))

Created dataset fiery-pipe-353815.insurance_demo


In [ ]:
client.delete_dataset(
    'insurance_demo', delete_contents=True, not_found_ok=True
) 
print("Deleted dataset insurance_demo")

Deleted dataset insurance_demo


In [ ]:
%%bigquery --project $PROJECT

CREATE OR REPLACE MODEL
    `insurance_demo.bqml_storm_clusters` OPTIONS(model_type='kmeans', num_clusters=6)
AS

WITH zipcodes
AS
(
SELECT
    state,
    zip_code,
    event_type,
    COUNT(event_id) no_of_storms,
    SUM(damage_property) AS damage_property,
    SUM(damage_crops) AS damage_crops,
    SUM(damage_property + damage_crops) AS damage_cost
  FROM
    `bigquery-public-data.noaa_historic_severe_storms.storms_*`,
    `bigquery-public-data.geo_us_boundaries.zip_codes` AS zipcodes
  WHERE
    EXTRACT(YEAR
    FROM
      event_begin_time)>= EXTRACT(YEAR
    FROM
      CURRENT_DATE())-5
    AND ST_WITHIN(event_point,zip_code_geom)
    AND state IN ('California','Texas','Louisiana','Georgia','Florida')  
  GROUP BY
    state,
    zip_code,
    event_type
)
SELECT event_type, no_of_storms, damage_property, damage_crops
FROM zipcodes



In [ ]:
%%bigquery --project $PROJECT

SELECT
  *
FROM
  ML.EVALUATE(MODEL `insurance_demo.bqml_storm_clusters`)

In [ ]:
%%bigquery --project $PROJECT

SELECT
  *
FROM
  ML.EVALUATE(MODEL `insurance_demo.bqml_storm_clusters`
,
    (
    SELECT
    state,
    zip_code,
    event_type,
    COUNT(event_id) no_of_storms,
    SUM(damage_property) AS damage_property,
    SUM(damage_crops) AS damage_crops,
    SUM(damage_property + damage_crops) AS damage_cost
  FROM
    `bigquery-public-data.noaa_historic_severe_storms.storms_*`,
    `bigquery-public-data.geo_us_boundaries.zip_codes` AS zipcodes
  WHERE
    EXTRACT(YEAR
    FROM
      event_begin_time)>= EXTRACT(YEAR
    FROM
      CURRENT_DATE())-5
    AND ST_WITHIN(event_point,zip_code_geom)
    AND state IN ('Georgia')  
  GROUP BY
    state,
    zip_code,
    event_type)
             )

In [ ]:
%%bigquery predicted --project $PROJECT

SELECT
  CENTROID_ID,
  state,
  zip_code,
  sum(no_of_storms) no_of_storms,
  sum(damage_property) damage_property,
  sum(damage_crops) damage_crops,
  sum(damage_cost) damage_cost
FROM
  ML.PREDICT( MODEL insurance_demo.bqml_storm_clusters,
    (
   SELECT
    state,
    zip_code,
    event_type,
    COUNT(event_id) no_of_storms,
    SUM(damage_property) AS damage_property,
    SUM(damage_crops) AS damage_crops,
    SUM(damage_property + damage_crops) AS damage_cost
  FROM
    `bigquery-public-data.noaa_historic_severe_storms.storms_*`,
    `bigquery-public-data.geo_us_boundaries.zip_codes` AS zipcodes
  WHERE
    EXTRACT(YEAR
    FROM
      event_begin_time)>= EXTRACT(YEAR
    FROM
      CURRENT_DATE())-5
    AND ST_WITHIN(event_point,zip_code_geom)
    AND state IN ('California','Texas','Louisiana','Georgia','Florida')  
  GROUP BY
    state,
    zip_code,
    event_type
    )
    
            )
    GROUP BY  
        CENTROID_ID,
        state,
        zip_code

In [ ]:
predicted.head()

,CENTROID_ID,state,zip_code,no_of_storms,damage_property,damage_crops,damage_cost
0,4,Texas,78260,4,0.0,0.0,0.0
1,4,Georgia,31027,7,33000.0,0.0,0.0
2,3,Georgia,30906,18,630.0,630.0,1260.0
3,4,Texas,77316,3,10000.0,3000.0,13000.0
4,6,California,94024,3,0.0,0.0,0.0
